In [21]:
import datasets

In [22]:
dataset = datasets.load_dataset("lerobot/aloha_sim_insertion_human_image")

In [23]:
dataset['train'][0]
dataset['train'].features

{'observation.images.top': Image(mode=None, decode=True, id=None),
 'observation.state': Sequence(feature=Value(dtype='float32', id=None), length=14, id=None),
 'action': Sequence(feature=Value(dtype='float32', id=None), length=14, id=None),
 'episode_index': Value(dtype='int64', id=None),
 'frame_index': Value(dtype='int64', id=None),
 'timestamp': Value(dtype='float32', id=None),
 'next.done': Value(dtype='bool', id=None),
 'index': Value(dtype='int64', id=None)}

In [24]:


l = dataset.remove_columns("observation.state")
l["train"].features


{'observation.images.top': Image(mode=None, decode=True, id=None),
 'action': Sequence(feature=Value(dtype='float32', id=None), length=14, id=None),
 'episode_index': Value(dtype='int64', id=None),
 'frame_index': Value(dtype='int64', id=None),
 'timestamp': Value(dtype='float32', id=None),
 'next.done': Value(dtype='bool', id=None),
 'index': Value(dtype='int64', id=None)}

In [25]:
from datasets.features import Value, Sequence, Image, Features

GELLO dataset to Lerobot dataset


gello

/episodes/
    episodeX/
        pickleY -> dict of numpy arrays.


should add following dict keys:
- episode_index
- frame_index
- next.done
- timestamp (how? simply assume fixed fps?)
- index?

In [26]:
#!wget https://cloud.ilabt.imec.be/index.php/s/2KWj4XJFzkbLbRE/download/planar_push_gello.zip


In [27]:
#!unzip planar_push_gello.zip

In [28]:
from pathlib import Path
import pickle 
import numpy as np 
from PIL import Image as PILImage

data_dir = Path("planar_push_gello")
episode_paths = list(data_dir.glob("*"))
episode_paths = sorted(episode_paths)
episode_paths

step_path = list(episode_paths[0].glob("*.pkl"))[0]
step_dict = pickle.load(open(step_path, "rb"))

dataset_keys = list(step_dict.keys())

gello_dataset_features = {}
for key in dataset_keys:
    print(key)
    print(type(step_dict.get(key)))
    if type(step_dict.get(key)) == np.ndarray:
        print(step_dict.get(key).shape)
        if len(step_dict.get(key).shape) ==0:
            if  type(step_dict.get(key)) == np.float64 or type(step_dict.get(key)) == np.float32:
                gello_dataset_features[key] = Value("float64")
            elif type(step_dict.get(key)) == np.int64 or type(step_dict.get(key)) == np.int32 or type(step_dict.get(key)) == np.int16:
                gello_dataset_features[key] = Value("int64")

        elif len(step_dict.get(key).shape) ==1:
            if  type(step_dict.get(key)[0]) == np.float64 or type(step_dict.get(key)[0]) == np.float32:
                gello_dataset_features[key] = Sequence(Value("float64"),length = len(step_dict.get(key)))
            elif type(step_dict.get(key)[0]) == np.int64 or type(step_dict.get(key)[0]) == np.int32 or type(step_dict.get(key)[0]) == np.int16:
                gello_dataset_features[key] = Sequence(Value("int64"),length = len(step_dict.get(key)))

        elif len(step_dict.get(key).shape) ==3:
            gello_dataset_features[key] = Image()
        
        else:
            print("could not find type for key")
            print(key, type(step_dict.get(key)))

    elif type(step_dict.get(key)) == list:
        if  type(step_dict.get(key)[0]) in [ np.float64, np.float32, np.float16, float]:
            gello_dataset_features[key] = Sequence(Value("float64"),length = len(step_dict.get(key)))
        elif type(step_dict.get(key)[0]) == np.int64 or type(step_dict.get(key)[0]) == np.int32 or type(step_dict.get(key)[0]) == np.int16 or type==int:
            gello_dataset_features[key] = Sequence(Value("int64"),length = len(step_dict.get(key)))
        else:
            print("could not find type for key")
            print(key, type(step_dict.get(key)[0]))
    
    elif type(step_dict.get(key)) == float:
        gello_dataset_features[key] = Value("float64")
    
    elif type(step_dict.get(key)) == int:
        gello_dataset_features[key] = Value("int64")
    
    else: 
        print("could not find type for key")
        print(key, type(step_dict.get(key)))

dataset_features = gello_dataset_features.copy()
dataset_features["frame_index"] = Value("int64")
dataset_features["episode_index"] = Value("int64")
dataset_features["index"] = Value("int64")
dataset_features["next.done"] = Value("int64")
dataset_features["next.success"] = Value("int64")
dataset_features["timestep"] = Value("float64")

dataset_features

wrist_rgb
<class 'numpy.ndarray'>
(480, 640, 3)
wrist_depth
<class 'numpy.ndarray'>
(480, 640, 1)
base_rgb
<class 'numpy.ndarray'>
(480, 640, 3)
base_depth
<class 'numpy.ndarray'>
(480, 640, 1)
joint_positions
<class 'numpy.ndarray'>
(6,)
tcp_pose_quat
<class 'list'>
gripper_position
<class 'numpy.ndarray'>
(1,)
wrench
<class 'list'>
control
<class 'numpy.ndarray'>
(6,)


{'wrist_rgb': Image(mode=None, decode=True, id=None),
 'wrist_depth': Image(mode=None, decode=True, id=None),
 'base_rgb': Image(mode=None, decode=True, id=None),
 'base_depth': Image(mode=None, decode=True, id=None),
 'joint_positions': Sequence(feature=Value(dtype='float64', id=None), length=6, id=None),
 'tcp_pose_quat': Sequence(feature=Value(dtype='float64', id=None), length=6, id=None),
 'gripper_position': Sequence(feature=Value(dtype='float64', id=None), length=1, id=None),
 'wrench': Sequence(feature=Value(dtype='float64', id=None), length=6, id=None),
 'control': Sequence(feature=Value(dtype='float64', id=None), length=6, id=None),
 'frame_index': Value(dtype='int64', id=None),
 'episode_index': Value(dtype='int64', id=None),
 'index': Value(dtype='int64', id=None),
 'next.done': Value(dtype='int64', id=None),
 'next.success': Value(dtype='int64', id=None),
 'timestep': Value(dtype='float64', id=None)}

In [88]:
import torch
dataset_dict = {}
for key in dataset_features:
    dataset_dict[key] = []

dataset_idx = 0
for episode_idx, episode_path in enumerate(episode_paths):
    step_paths = list(episode_path.glob("*.pkl"))
    step_paths = sorted(step_paths)
    for step_idx, step_path in enumerate(step_paths):
        step_dict = pickle.load(open(step_path, "rb"))

        step_dict["frame_index"] = step_idx
        step_dict["episode_index"] = episode_idx
        step_dict["index"] = dataset_idx

        # assume all demonstrations are successful at the last step. 
        step_dict["next.done"] = len(step_paths) -1 == step_idx
        step_dict["next.success"] = step_dict["next.done"] 

        step_dict["timestep"] = step_idx * 0.1 # todo: fix hardcoded fps, can parse from filenames.

        for key in gello_dataset_features:
            if "rgb" in key or "depth" in key:
                # type to uint8 from uint16
                step_dict[key] = step_dict[key].astype(np.uint8)
                if "depth" in key:
                    step_dict[key] = step_dict[key][...,0] # (H,W) instead of (1,H,W) for pillow
                
                step_dict[key] = PILImage.fromarray(step_dict[key])
            else:
                step_dict[key] = torch.tensor(step_dict.get(key))

        for key,value in step_dict.items():
            dataset_dict[key].append(value)

        dataset_idx += 1
        


In [89]:
for key in dataset_dict:
    if type(dataset_dict[key][0]) == torch.Tensor:
        print(key)
        dataset_dict[key] = torch.stack(dataset_dict[key])

joint_positions
tcp_pose_quat
gripper_position
wrench
control


In [90]:
hf_dataset = datasets.Dataset.from_dict(dataset_dict, features=Features(dataset_features))

wrist_rgb
wrist_depth
base_rgb
base_depth
joint_positions
tcp_pose_quat
gripper_position
wrench
control
frame_index
episode_index
index
next.done
next.success
timestep


In [78]:
type(hf_dataset[0]["joint_positions"])

list

In [91]:
# change control to "action"
if "control" in hf_dataset.column_names:
    hf_dataset = hf_dataset.rename_column("control", "action")

# concatenate two columns into one
if not "observation.state" in hf_dataset.column_names:  
    hf_dataset = hf_dataset.map(lambda x: {"observation.state": x["joint_positions"] + x["gripper_position"]})


# rename "wrist_rgb" to "observation.image.wrist" and "base_rgb" to "observation.image.base"

if "wrist_rgb" in hf_dataset.column_names:
    hf_dataset = hf_dataset.rename_column("wrist_rgb", "observation.image.wrist")

if "base_rgb" in hf_dataset.column_names:
    hf_dataset = hf_dataset.rename_column("base_rgb", "observation.image.base")

# drop wrist_depth and base_depth 
if "wrist_depth" in hf_dataset.column_names:
    hf_dataset = hf_dataset.remove_columns("wrist_depth")

if "base_depth" in hf_dataset.column_names:
    hf_dataset = hf_dataset.remove_columns("base_depth")


# 

Map: 100%|██████████| 9717/9717 [00:09<00:00, 1060.59 examples/s]


In [72]:
hf_dataset[0]["observation.state"]

[-0.004509274159566701,
 -1.1877484780601044,
 1.68516713777651,
 -2.0681268177428187,
 -1.570770565663473,
 -0.004645172749654591,
 -0.004645172749654591]

In [1]:
# push the dataset to the hub
hf_dataset.push_to_hub("tlpss/planar-push-gello",split="train")

NameError: name 'hf_dataset' is not defined

In [9]:
from datasets import load_dataset

dataset = load_dataset("tlpss/planar-push-gello", split="train",revision="v1.4")

In [11]:
# change timestamps by dividing by 100
dataset = dataset.map(lambda x: {"timestamp": x["timestamp"] / 100.0})


In [12]:
dataset[1]

{'tcp_pose_quat': [-0.6034055948257446,
  -0.12928718328475952,
  0.00946362316608429,
  -2.2129580974578857,
  -2.223250150680542,
  -0.004154943395406008],
 'wrench': [18.356094360351562,
  25.67031478881836,
  -34.85917282104492,
  -0.6768204569816589,
  0.6910861134529114,
  -0.015712976455688477],
 'frame_index': 1,
 'episode_index': 0,
 'index': 1,
 'timestamp': 0.10000000149011612,
 'next.done': False,
 'next.success': False,
 'observation.state': [-0.009812959469854832,
  -1.193248987197876,
  1.6807794570922852,
  -2.073329448699951,
  -1.5757499933242798,
  -0.009967454709112644],
 'action': [-0.005287617444992065,
  -1.1893846988677979,
  1.6874358654022217,
  -2.068847417831421,
  -1.5707963705062866,
  -0.005287617444992065],
 'observation.images.wrist.rgb': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x480>,
 'observation.images.base.rgb': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x480>}

In [13]:
# upload the changes to the hub

dataset.push_to_hub("tlpss/planar-push-gello",split="train")

Uploading the dataset shards: 100%|██████████| 13/13 [03:03<00:00, 14.10s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/tlpss/planar-push-gello/commit/50c767f70013e60162751d97d5b646302af6bd81', commit_message='Upload dataset', commit_description='', oid='50c767f70013e60162751d97d5b646302af6bd81', pr_url=None, pr_revision=None, pr_num=None)